# Wisconsin

First import the packages that will be needed to access and clean the data. This will require a web browser to be launched with selenium. For this project I used geckodriver. The pyautogui is used to download the look up table. After clicking the search button, all of the wisconsin schools appear and selenium is not able to find the download button in the viewport. Using the scroll functionality of pyautogui brings the download button into view.

In [31]:
import datetime
import time
import wget
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import zipfile
from geopy.geocoders import Nominatim
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pyautogui

In [2]:
ThisYear = str(datetime.datetime.now().year - 1)  + '-' + str(datetime.datetime.now().year)[2:]
PreString = 'https://dpi.wi.gov/sites/default/files/wise/downloads/enrollment_certified_'
PostString = '.zip'
File = PreString + ThisYear + PostString
File

'https://dpi.wi.gov/sites/default/files/wise/downloads/enrollment_certified_2017-18.zip'

In [3]:
ThisYearFile = wget.download(File)

100% [............................................................................] 749325 / 749325

In [4]:
LastYear = str(datetime.datetime.now().year - 2)  + '-' + str(datetime.datetime.now().year -1)[2:]
File = PreString + LastYear + PostString
LastYearFile = wget.download(File)

100% [............................................................................] 749421 / 749421

In [5]:
zf = zipfile.ZipFile(ThisYearFile) # having First.csv zipped file.
WIThYr = pd.read_csv(zf.open('enrollment_certified_' + ThisYear +'.csv'))

In [6]:
zf = zipfile.ZipFile(LastYearFile) # having First.csv zipped file.
WILsYr = pd.read_csv(zf.open('enrollment_certified_' + LastYear + '.csv'))

In [7]:
WILsYr.columns

Index(['SCHOOL_YEAR', 'AGENCY_TYPE', 'CESA', 'COUNTY', 'DISTRICT_CODE',
       'SCHOOL_CODE', 'GRADE_GROUP', 'CHARTER_IND', 'DISTRICT_NAME',
       'SCHOOL_NAME', 'GROUP_BY', 'GROUP_BY_VALUE', 'STUDENT_COUNT',
       'PERCENT_OF_GROUP'],
      dtype='object')

In [8]:
WILsYr = WILsYr[(WILsYr.GROUP_BY == 'Grade Level') & (WILsYr.GRADE_GROUP != '[All]') &
((WILsYr.GROUP_BY_VALUE == '11') | (WILsYr.GROUP_BY_VALUE == '12'))]

In [9]:
WIThYr = WIThYr[(WIThYr.GROUP_BY == 'Grade Level') & (WIThYr.GRADE_GROUP != '[All]') &
((WIThYr.GROUP_BY_VALUE == '11') | (WIThYr.GROUP_BY_VALUE == '12'))]

In [10]:
WILsYr.columns

Index(['SCHOOL_YEAR', 'AGENCY_TYPE', 'CESA', 'COUNTY', 'DISTRICT_CODE',
       'SCHOOL_CODE', 'GRADE_GROUP', 'CHARTER_IND', 'DISTRICT_NAME',
       'SCHOOL_NAME', 'GROUP_BY', 'GROUP_BY_VALUE', 'STUDENT_COUNT',
       'PERCENT_OF_GROUP'],
      dtype='object')

In [11]:
WILsYr.GROUP_BY_VALUE.drop_duplicates()

203    11
204    12
Name: GROUP_BY_VALUE, dtype: object

In [12]:
len(WILsYr[WILsYr['GROUP_BY_VALUE'] == '12'])

579

In [13]:
# Separate grades into different dfs for last year
WILsYrSr = WILsYr[WILsYr['GROUP_BY_VALUE'] == '12']
WIThYrSr = WIThYr[WIThYr['GROUP_BY_VALUE'] == '12']

In [14]:
# Separate grades into different dfs for this year
WILsYrJr = WILsYr[WILsYr['GROUP_BY_VALUE'] == '11']
WIThYrJr = WIThYr[WIThYr['GROUP_BY_VALUE'] == '11']

In [54]:
columnsThsYr = ['DISTRICT_CODE', 'SCHOOL_CODE', 'SCHOOL_NAME', 'STUDENT_COUNT', 'DISTRICT_NAME']
columnsLstYr = ['DISTRICT_CODE', 'SCHOOL_CODE', 'STUDENT_COUNT']

In [55]:
WIThYrJr.columns

Index(['SCHOOL_YEAR', 'AGENCY_TYPE', 'CESA', 'COUNTY', 'DISTRICT_CODE',
       'SCHOOL_CODE', 'GRADE_GROUP', 'CHARTER_IND', 'DISTRICT_NAME',
       'SCHOOL_NAME', 'GROUP_BY', 'GROUP_BY_VALUE', 'STUDENT_COUNT',
       'PERCENT_OF_GROUP'],
      dtype='object')

In [56]:
# Combine Jr and Sr as separate columns instead of rows
WIL = pd.merge(WILsYrSr[columnsThsYr], WILsYrJr[columnsLstYr], how = 'inner', on = ['DISTRICT_CODE', 'SCHOOL_CODE'])
WIT = (pd.merge(WIThYrSr[columnsThsYr], WIThYrJr[columnsLstYr], how = 'inner', on = ['DISTRICT_CODE', 'SCHOOL_CODE']))

In [57]:
WIL.columns

Index(['DISTRICT_CODE', 'SCHOOL_CODE', 'SCHOOL_NAME', 'STUDENT_COUNT_x',
       'DISTRICT_NAME', 'STUDENT_COUNT_y'],
      dtype='object')

In [58]:
WIL = WIL.rename(columns = {'STUDENT_COUNT_x':'Juniors Last Year',
                       'STUDENT_COUNT_y': 'Seniors Last Year' })

In [59]:
WIT = WIT.rename(columns = {'STUDENT_COUNT_x':'Juniors This Year',
                       'STUDENT_COUNT_y': 'Seniors This Year' })

In [60]:
columnsThsYr = ['DISTRICT_CODE', 'SCHOOL_CODE', 'SCHOOL_NAME', 'DISTRICT_NAME', 'Juniors This Year', 'Seniors This Year']
columnsLstYr = ['DISTRICT_CODE', 'SCHOOL_CODE',  'Juniors Last Year', 'Seniors Last Year']

In [61]:
WI = pd.merge(WIL[columnsLstYr], WIT[columnsThsYr], how = 'inner', on = ['DISTRICT_CODE', 'SCHOOL_CODE'])

In [62]:
#WI['Junior Delta'] 

In [63]:
len(WI)

554

In [25]:
# Set the preferences for the firefox web browser
fp = webdriver.FirefoxProfile()
fp.set_preference('browser.download.folderList', 2)
fp.set_preference('browser.download.manager.showWhenStarting', False)
fp.set_preference('browser.download.dir', 'C:\\Users\\karlk\\GitHubRepos\\SCSU-Reciprocity-HS-Enrollment\\Wisconsin')
fp.set_preference("http.response.timeout", 300)
fp.set_preference("dom.max_script_run_time", 300)
fp.set_preference('webdriver.load.strategy', 'unstable')
fp.set_preference('browser.helperApps.neverAsk.saveToDisk', 'text/plain, application/vnd.ms-excel, text/csv, text/comma-separated-values, application/octet-stream, application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
fp.update_preferences()

In [26]:
# MDE web app for hs enrollment data
searchURL = "https://apps4.dpi.wi.gov/SchoolDirectory/Search/PublicSchoolsSearch"
time.sleep(3)

# Launch web browser and navigate to the searchURL
driver = webdriver.Firefox(fp)
driver.get(searchURL)
time.sleep(3)

In [64]:
WI.head()

,DISTRICT_CODE,SCHOOL_CODE,Juniors Last Year,Seniors Last Year,SCHOOL_NAME,DISTRICT_NAME,Juniors This Year,Seniors This Year
0,7,40.0,45,52,Abbotsford Middle/Senior High,Abbotsford,49,58
1,14,40.0,133,103,Adams-Friendship High,Adams-Friendship Area,101,135
2,63,40.0,28,31,Albany High,Albany,29,24
3,70,40.0,70,65,Algoma High,Algoma,61,59
4,84,40.0,16,12,Alma High,Alma,14,21


In [30]:
Search = driver.find_element_by_id("txtSearch")
searchbutton = driver.find_element_by_xpath("//input[@value='Search']")

time.sleep(1)
time.sleep(1)
ActionChains(driver).move_to_element(searchbutton).click().perform()

time.sleep(3)

In [48]:
pyautogui.moveTo(x=50, y=100)
pyautogui.click() 
time.sleep(1)
for i in range(200):
    pyautogui.scroll(-100000000) 

In [49]:
downloadbutton = driver.find_element_by_xpath("//input[@value='Download']")
ActionChains(driver).move_to_element(downloadbutton).click().perform()

In [50]:
WILookup = pd.read_csv('Directory.csv')

In [65]:
WILookup.head()

,Public School Name,District Name,LEA Code,CESA,School Code,School Type,Low Grade,High Grade,Contact,Contact Title,...,Extension,Fax,Email,Website,School Category,Charter,Virtual,District NCES,School NCES,Agency Type Description
0,21st Century eSchool,Middleton-Cross Plains Area,3549,2,800,Combined Elementary/Secondary School,KG,12,Jill Gurtner,Principal,...,NaN,6.088315e+09,jgurtner@mcpasd.k12.wi.us,www.mcpasd.k12.wi.us,Regular,Charter,Exclusively Virtual,5509510.0,2818.0,Public school
1,21st Century Prep Sch,21st Century Prep Sch,8110,1,100,Elementary School,K4,08,Valencia Koker,Executive Director,...,1170.0,2.625980e+09,vkoker@tcpswi.org,www.21stprepschool.org,Regular,Charter,Primarily Classroom (Supplemental Virtual),5500045.0,2575.0,Non District Charter Schools
2,4K Center for Literacy,Kimberly Area,2835,6,150,Elementary School,K4,PK,Sean Fitzgerald,Principal,...,NaN,9.204234e+09,kasd@kimberly.k12.wi.us,www.kimberly.k12.wi.us,Regular,No,Primarily Classroom (Supplemental Virtual),5507470.0,3029.0,Public school
3,4K Community Sch,Burlington Area,777,2,150,Elementary School,K4,K4,Connie Zinnen,Assistant Superintendent of Instruction,...,NaN,2.627630e+09,czinnen@basd.k12.wi.us,www.basd.k12.wi.us,Regular,No,No,5501890.0,2987.0,Public school
4,4K Community-Based,Reedsburg,4753,5,150,Elementary School,K4,K4,Paul Bierman,Principal,...,2028.0,6.087689e+09,pbierman@rsd.k12.wi.us,www.rsd.k12.wi.us,Regular,No,Primarily Classroom (Supplemental Virtual),5512660.0,2889.0,Public school


In [52]:
WILookup.columns

Index(['Public School Name', 'District Name', 'LEA Code', 'CESA',
       'School Code', 'School Type', 'Low Grade', 'High Grade', 'Contact',
       'Contact Title', 'Physical Address', 'Mailing Address', 'Mailing City',
       'Mailing Zip', 'Mailing Zip + 4', 'Shipping Address', 'Physical City',
       'County Code', 'County Name', 'Phone', 'Extension', 'Fax', 'Email',
       'Website', 'School Category', 'Charter', 'Virtual', 'District NCES',
       'School NCES', 'Agency Type Description'],
      dtype='object')

In [66]:
WI.columns

Index(['DISTRICT_CODE', 'SCHOOL_CODE', 'Juniors Last Year',
       'Seniors Last Year', 'SCHOOL_NAME', 'DISTRICT_NAME',
       'Juniors This Year', 'Seniors This Year'],
      dtype='object')

In [69]:
Full = pd.merge(WI, WILookup, how = 'inner', left_on = ['DISTRICT_NAME', 'SCHOOL_CODE']
        , right_on = ['District Name', 'School Code'])

In [70]:
len(Full)

549

In [71]:
Full.to_csv('FullWI.csv', index = False)